In [39]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from concurrent.futures import ProcessPoolExecutor
import os

In [13]:
# funtion that computes the ij entry of the kronecker product of A and B

def kronecker(A,B,i,j):
    return A[i//B.shape[0],j//B.shape[1]]*B[i%B.shape[0],j%B.shape[1]]

# function that finds neigbors in the kronecker product of A and B

def neighbors_kron(A,B,i):
    n = []
    for j in range(A.shape[1]*B.shape[1]):
        if kronecker(A,B,i,j) != 0:
            n.append(j)
    return n

In [29]:
# funtion that uses GGRF
def vector_rf_kron(W1, W2, d, f_vec, p_h, node, random_walks = 100, h = 100):
    '''
    This funtion computes the feature vector of a node using GGRF
    Args:
        W1: Adjacency matrix of the first graph
        W2: Adjacency matrix of the second graph
        d: Degree vector
        f_vec: Function to compute modulation of the random walk
        p_h: Probability of stopping the random walk
        node: Node of interest
        random_walks: Number of random walks
        h: Default value
    Returns:
        phi: Feature vector of the node
    '''
    # Initial values
    n = h
    phi = np.zeros(len(d))
    m = random_walks
    f_m = f_vec(n)

    for w in range(m):
        # Initial values for the random walk
        load = 1
        current_node = node
        terminated = False
        walk_lenght = 0
        
        # Register of the nodes visited
        register = [current_node]
        counter = 0
        while terminated == False:
            
            # In case we require more values of f
            if walk_lenght == n:
                #print("Requerí mas valores de f")
                n = 2 * n
                f_m = f_vec(n)

            # Update the feature vector
            phi[current_node] += load * f_m[walk_lenght]
            # Update the walk length
            walk_lenght += 1

            # Select the next node searching in the neighbors
            neighbors = neighbors_kron(W1,W2,current_node)
            new_node = np.random.choice(neighbors)
            aux = []
            # If the node is already in the register, we search for a new one
            while new_node in register:
                aux.append(new_node)
                new_node = np.random.choice(neighbors)
                if len(aux) == len(neighbors):
                    break
            # If we tried all the neighbors, we select a random one
            if len(aux) == len(neighbors):
                new_node = np.random.choice(neighbors)

            # Update the load
            load = load * (d[current_node] / (1 - p_h))* kronecker(W1,W2,current_node,new_node)

            # Update the current node
            current_node = new_node

            # Update the register
            register.append(current_node)
            counter += 1

            # Check if the random walk is terminated
            terminated = (np.random.uniform(0,0.5) < p_h)
            if counter == 150:
                break

    return phi / m

In [27]:
# modulation function
def compute_f_vector(f_alpha, n):
    '''
    This function computes the modulation function for a given alpha function and n
    according to the GGRF paper
    Args:
        f_alpha: Alpha function
        n: Number of values to compute
    Returns:
        f: Modulation function of length n
    '''
    alpha = f_alpha(n)
    f = np.zeros(n)

    # Initial values
    f[0] = np.sqrt(alpha[0])
    aux = 2 * f[0]

    f[1] = alpha[1] / aux

    f[2] = (alpha[2] - f[1]**2) / aux

    # Compute the rest of the values
    for i in range(3, n):
        suma = sum(f[i-p] * f[p] for p in range(1, i))
        f[i] = (alpha[i] - suma) / aux

    return f

In [28]:
# coefficients for a Laplacian kernel

def alpha_laplace(s, n, d = 1):
    '''
    This function computes the alpha function for a Laplacian kernel
    Args:
        s: Laplacian kernel parameter for regularization
        n: Number of values to compute
        d: Default value (power of the degree)
    Returns:
        alpha: Alpha function of length n
    '''
    alpha = np.ones(n)
    aux1 = 0
    aux2 = 1
    # Recurrent formula
    q = 1 / (1 + s**(-2))
    #q = 1

    for i in range(1, n):
        alpha[i] = ((d + aux1) / aux2) * q * alpha[i-1]
        aux1 += 1
        aux2 += 1

    return alpha

In [ ]:
def kernel_graph_random_features(W1, W2, dx, f_vec, Px, Qx, p_h, random_walks = 100):
    '''
    This function computes the kernel value using the random features method
    Args:
        W1: Adjacency matrix of the first graph
        W2: Adjacency matrix of the second graph
        dx: Degree vector
        f_vec: Function to compute modulation of the random walk
        Px: Probability vector with arriving probabilities
        Qx: Probability vector with leaving probabilities
        p_h: Probability of stopping the random walk
        random_walks: Number of random walks
    Returns:
        K: Kernel value
    '''
    # Define the matrices to store the feature vectors
    K1 = []
    K2 = []

    # Iteration over the nodes
    for i in range(len(dx)):
        # Compute the feature vector for the node i
        phi1 = vector_rf_kron(W1, W2, dx, f_vec, p_h, i, random_walks)
        K1.append(phi1)
        phi1 = vector_rf_kron(W1, W2, dx, f_vec, p_h, i, random_walks)
        K2.append(phi1)

    # Compute the estimation
    K = np.dot(K1, np.transpose(K2))

    return np.dot(Qx, np.dot(K, Px))

In [51]:
# function to compute the kernel matrix

def kernel_graph_random_features1(W1, W2, dx, f_vec, Px, Qx, p_h, random_walks = 100):
    '''
    This function computes the kernel value using the random features method with changes to
    save memory
    Args:
        W1: Adjacency matrix of the first graph
        W2: Adjacency matrix of the second graph
        dx: Degree vector
        f_vec: Function to compute modulation of the random walk
        Px: Probability vector with arriving probabilities
        Qx: Probability vector with leaving probabilities
        p_h: Probability of stopping the random walk
        random_walks: Number of random walks
    Returns:
        K: Kernel value
    '''
    # Initial value
    K = 0
    vectors = []
    # Compute and save the first vector
    phi_0 = vector_rf_kron(W1, W2, dx, f_vec, p_h, 0, random_walks)
    aux = np.zeros(len(phi_0))
    for i in range(W1.shape[1]*W2.shape[1]):
        phi_i = vector_rf_kron(W1, W2, dx, f_vec, p_h, i, random_walks)
        vectors.append(phi_i)
        aux[i] = np.dot(phi_0, phi_i)
    
    # Add the first term to the kernel
    K += Qx[0] * np.dot(Px, aux)

    # Compute the rest of the terms
    for i in range(1, W1.shape[1]*W2.shape[1]):
        phi_i = vector_rf_kron(W1, W2, dx, f_vec, p_h, i, random_walks)
        aux = np.zeros(len(phi_i))
        for j in range(W1.shape[1]*W2.shape[1]):
            aux[j] = np.dot(phi_i, vectors[j])
        # Add the term to the kernel
        K += Qx[i] * np.dot(Px, aux)

    return K

In [47]:
# function to compute the kernel matrix

def kernel_graph_random_features2(W1, W2, dx, f_vec, Px, Qx, p_h, random_walks = 100):
    '''
    This function computes the kernel value using the random features method savig the vectors
    to solve the ram problem
    Args:
        W1: Adjacency matrix of the first graph
        W2: Adjacency matrix of the second graph
        dx: Degree vector
        f_vec: Function to compute modulation of the random walk
        Px: Probability vector with arriving probabilities
        Qx: Probability vector with leaving probabilities
        p_h: Probability of stopping the random walk
        random_walks: Number of random walks
    Returns:
        K: Kernel value
    '''
    # Initial value
    K = 0
    # Create the directory to save the vectors
    os.makedirs('vectors', exist_ok=True)
    # Compute and save the first vector
    phi_0 = vector_rf_kron(W1, W2, dx, f_vec, p_h, 0, random_walks)
    aux = np.zeros(len(phi_0))
    for i in range(W1.shape[1]*W2.shape[1]):
        phi_i = vector_rf_kron(W1, W2, dx, f_vec, p_h, i, random_walks)
        np.save('vectors/phi_{}'.format(i), phi_i)
        aux[i] = np.dot(phi_0, phi_i)
    
    # Add the first term to the kernel
    K += Qx[0] * np.dot(Px, aux)

    # Compute the rest of the terms
    for i in range(1, W1.shape[1]*W2.shape[1]):
        phi_i = vector_rf_kron(W1, W2, dx, f_vec, p_h, i, random_walks)
        aux = np.zeros(len(phi_i))
        for j in range(W1.shape[1]*W2.shape[1]):
            # we only have to load the vectors from the disk
            phi_j = np.load('vectors/phi_{}.npy'.format(j))
            aux[j] = np.dot(phi_i, phi_j)
        # Add the term to the kernel
        K += Qx[i] * np.dot(Px, aux)
    # Remove the directory
    os.system('rm -r vectors')

    return K

In [40]:
def compute_phi_and_dot(phi_0, W1, W2, dx, f_vec, p_h, index, random_walks):
    """
    Compute phi_i and its dot product with phi_0
    """
    phi_i = vector_rf_kron(W1, W2, dx, f_vec, p_h, index, random_walks)
    return index, phi_i, np.dot(phi_0, phi_i)

def kernel_graph_random_features_parallel(W1, W2, dx, f_vec, Px, Qx, p_h, random_walks=100):
    '''
    Computes the kernel value using the random features method with parallelization
    Args:
        W1: Adjacency matrix of the first graph
        W2: Adjacency matrix of the second graph
        dx: Degree vector
        f_vec: Function to compute modulation of the random walk
        Px: Probability vector with arriving probabilities
        Qx: Probability vector with leaving probabilities
        p_h: Probability of stopping the random walk
        random_walks: Number of random walks
    Returns:
        K: Kernel value
    '''
    K = 0
    n = W1.shape[1] * W2.shape[1]
    
    # Compute phi_0
    phi_0 = vector_rf_kron(W1, W2, dx, f_vec, p_h, 0, random_walks)
    
    # Parallel computation of phi_i and dot(phi_0, phi_i)
    with ProcessPoolExecutor() as executor:
        futures = [
            executor.submit(compute_phi_and_dot, phi_0, W1, W2, dx, f_vec, p_h, i, random_walks)
            for i in range(n)
        ]
        results = [f.result() for f in futures]
    
    # Process results for the first term
    phi_dict = {index: phi_i for index, phi_i, dot_value in results}
    aux = np.array([dot_value for _, _, dot_value in results])
    K += Qx[0] * np.dot(Px, aux)

    # Compute the rest of the terms in parallel
    def compute_aux_and_contrib(phi_i, i):
        aux = np.zeros(n)
        for j in range(n):
            aux[j] = np.dot(phi_i, phi_dict[j])
        return Qx[i] * np.dot(Px, aux)

    with ProcessPoolExecutor() as executor:
        futures = [
            executor.submit(compute_aux_and_contrib, phi_dict[i], i)
            for i in range(1, n)
        ]
        results = [f.result() for f in futures]
    
    # Sum all contributions
    K += sum(results)

    return K


In [31]:
def count_trips_mibici(data_user, threshold = 5, complement = False):
    viajes_user = data_user.groupby([data_user[['Origen_Id', 'Destino_Id']].min(axis=1), data_user[['Origen_Id', 'Destino_Id']].max(axis=1)]).size().reset_index(name='counts')
    viajes_user.columns = ['Est_A', 'Est_B', 'counts']
    if not complement:
        viajes_user = viajes_user[viajes_user['counts'] >= threshold]
    else:
        viajes_user = viajes_user[viajes_user['counts'] < threshold]
    if viajes_user.empty:
        return None
    total = viajes_user['counts'].sum()
    viajes_user['prob'] = viajes_user['counts']/total
    viajes_user = viajes_user.sort_values(by = 'prob', ascending = False).reset_index(drop=True)
    return viajes_user

In [32]:
def compute_matrix(counter_user, normalized = False, self_loops = False):
    if not self_loops:
        counter_user = counter_user[counter_user['Est_A'] != counter_user['Est_B']]
    vertex = list(set(counter_user['Est_A'].unique().tolist() + counter_user['Est_B'].unique().tolist()))
    matrix = np.zeros((len(vertex), len(vertex)))
    for i in range(len(counter_user)):
        current_trip = counter_user.iloc[i]
        count = current_trip["counts"]
        estA = current_trip["Est_A"]
        estB = current_trip["Est_B"]

        matrix[vertex.index(estA)][vertex.index(estB)] = count
        matrix[vertex.index(estB)][vertex.index(estA)] = count
    if normalized:
        D = np.sum(matrix, axis = 1)
        D = np.diag(D)
        D = np.linalg.inv(np.sqrt(D))
        matrix = np.sqrt(D) @ matrix @ np.sqrt(D)
    return matrix

In [33]:
# function to compute the degree vector of a kronecker product

def degree_kron(W1, W2):
    return np.kron(W1.sum(axis = 1), W2.sum(axis = 1))

In [34]:
#dir = '/home/user/Desktop/Datos/'
dir = '/Users/antoniomendez/Desktop/Tesis/Datos/datos_limpios/'

In [35]:
data_2019 = pd.read_csv(f'{dir}mibici/2019.csv')
data = data_2019[data_2019['Inicio_del_viaje'].str.startswith('2019-01-01')]
data2 = data_2019[data_2019['Inicio_del_viaje'].str.startswith('2019-01-02')]

In [36]:
counts_data = count_trips_mibici(data)
counts_data2 = count_trips_mibici(data2)

In [ ]:
m1 = compute_matrix(counts_data, normalized = True)
m2 = compute_matrix(counts_data2, normalized = True)

In [49]:
dx = degree_kron(m1, m2)

px = np.ones(len(dx)) / len(dx)
qx = np.ones(len(dx)) / len(dx)

f_alpha = lambda n: alpha_laplace(0.1, n, 1)

f_vec = lambda n: compute_f_vector(f_alpha, n)

p_h = 0.15

K = kernel_graph_random_features(m1, m2, dx, f_vec, px, qx, p_h)
print(K)

0.0002389622548040926


In [50]:
import requests
def send_message(message, channel):
    requests.post(f"https://ntfy.sh/{channel}",
        data=message.encode(encoding='utf-8'))
    
send_message(f"Kernel value: {K}", "My_Computer")